In [3]:
#glider_zooplankton_regressions.ipynb
#a script to open up colocated data and calculate linear regressions
import pandas as pd
import tqdm

from scipy.stats import linregress
import numpy as np



base = '/Users/jeffdorman/Library/CloudStorage/GoogleDrive-jdorman@faralloninstitute.org/'
project = 'Shared drives/Commons/PROJECTS/2024_IOOS_GliderZooplankton/'
data = 'data/Colocated_Glider-Zooscan_VaryingSpatialTemporalScales/'


v1 ='appendicularia Estimated C Biomass (mgC m-2)'
v2 ='bryozoan_larvae Estimated C Biomass (mgC m-2)' #not assigned
v3 ='chaetognatha Estimated C Biomass (mgC m-2)'    #not assigned
v4 ='cnidaria_ctenophores Estimated C Biomass (mgC m-2)'
v5 ='copepoda_calanoida Estimated C Biomass (mgC m-2)'
v6 ='copepoda_eucalanids Estimated C Biomass (mgC m-2)'
v7 ='copepoda_harpacticoida Estimated C Biomass (mgC m-2)'
v8 ='copepoda_oithona_like Estimated C Biomass (mgC m-2)'
v9 ='copepoda_others Estimated C Biomass (mgC m-2)'
v10='copepoda_poecilostomatoids Estimated C Biomass (mgC m-2)'
v11='crustacea_others Estimated C Biomass (mgC m-2)'
v12='doliolids Estimated C Biomass (mgC m-2)'      
v13='eggs Estimated C Biomass (mgC m-2)'           #not assigned*
v14='euphausiids Estimated C Biomass (mgC m-2)'
v15='multiples Estimated C Biomass (mgC m-2)'      #not assigned
v16='nauplii Estimated C Biomass (mgC m-2)'
v17='ostracods Estimated C Biomass (mgC m-2)'
v18='others Estimated C Biomass (mgC m-2)'        #not assigned*
v19='polychaete Estimated C Biomass (mgC m-2)'    #not assigned*
v20='pteropoda Estimated C Biomass (mgC m-2)'
v21='pyrosomes Estimated C Biomass (mgC m-2)'
v22='rhizaria Estimated C Biomass (mgC m-2)'      #not assigned*
v23='salps Estimated C Biomass (mgC m-2)'


spatial_list = []
temporal_list = []
rsquared_list = []
intercept_list = []
slope_list = []
len_list = []

yvar='biomass'
#yvar='copepod'
#yvar='krill'
#yvar='pteropod'
#yvar='gelatinous'
#yvar='other'

for spatial_buffer in tqdm.tqdm([1, 5, 10, 15, 20]):
    for temporal_buffer in [0.5, 1, 7, 14, 30]:
        f='Backscatter_vs_Biomass_' + str(temporal_buffer) + 'day_' + str(spatial_buffer) +  'km.csv'
        fname= base + project + data + f
        d = pd.read_csv(fname)
        x=d['big_sv']
        flag=d['acoustic_backscatter_flag']
        #match yvar:
        #    case "biomass":
        #        y=d['total_biomass']
        #    case "copepod"
        #        y=(d[v5]+d[v6]+d[v7]+d[v8]+d[v9]+d[v10]+d[v11]+d[v16]+d[v17])
        #    case 'krill'
        #        y=d[v14]
        #    case 'pteropod'
        #        y=d[v20]
        #    case 'gelatinous'
        #        y=(d[v1]+d[v4]+d[v12]+d[v21]+d[v23])
        #    case 'other'
        #        y=(d[v13]+d[v18]+d[v19]+d[v22])
        #    case _:
        #        print("No Y Varaible has been defined.")
        
        y=d['total_biomass']

        #Elegant way to deal with the sensor (750 vs 1000) that didn't work       
        #sensor=750
        #if sensor==750:
        #    mask = ~np.isnan(x) & ~np.isnan(y) & flag==0
        #elif sensor==1000:
        #    mask = ~np.isnan(x) & ~np.isnan(y) & flag==1
        #else:
        
        #Two Step it:  First step to get rid of NaN's in x and y
        mask = ~np.isnan(x) & ~np.isnan(y)
        xx=x[mask]
        yy=y[mask]
        ff=flag[mask]
        
        
        sensor_specific = 0  #Set to 1 to turn on sensor specific model
        #Second step to filter by 750 (flag 0) or 1000 (flag 1) kHz sensors
        if sensor_specific == 1:
            sensor=750
            if sensor == 750:
                loar=ff==0
            elif sensor == 1000:
                loar=ff==1
            
            xxx=xx[loar]
            yyy=yy[loar]
            fff=ff[loar]
            slope, intercept, r_value, p_value, std_err = linregress(xxx, np.log10(yyy+1))
            len_list.append(len(xxx))
        else:
            slope, intercept, r_value, p_value, std_err = linregress(xx, np.log10(yy+1))
            len_list.append(len(xx))

        #add values to lists
        spatial_list.append(spatial_buffer)
        temporal_list.append(temporal_buffer)
        rsquared_list.append(r_value*r_value)
        intercept_list.append(intercept)
        slope_list.append(slope)

results = pd.DataFrame({'Spatial':spatial_list, 'Temporal':temporal_list, 'RSquared':rsquared_list, 'Slope':slope_list, 'Intercept':intercept_list, 'N':len_list})
        
sv=0
if sv==1:
    results.to_csv(base + project + data + 'Backscatter_vs_EuphausiidBiomass_RegressionResults.csv')

results



100%|█████████████████████████████████████████████| 5/5 [00:00<00:00, 12.22it/s]


,Spatial,Temporal,RSquared,Slope,Intercept,N
0,1,0.5,0.487942,0.045276,2.807272,9
1,1,1.0,0.600863,0.071454,2.409854,13
2,1,7.0,0.273410,0.057011,2.596995,75
3,1,14.0,0.171813,0.042053,2.770040,143
4,1,30.0,0.144312,0.037393,2.810625,246
5,5,0.5,0.222709,0.039721,2.778373,46
6,5,1.0,0.225751,0.037541,2.795202,73
7,5,7.0,0.213910,0.038361,2.777827,351
8,5,14.0,0.156189,0.032198,2.845832,586
9,5,30.0,0.161042,0.034396,2.798282,815


In [4]:
results.pivot(index='Temporal', columns='Spatial', values='RSquared')




Spatial,1,5,10,15,20
Temporal,,,,,
0.5,0.487942,0.222709,0.291593,0.263726,0.251455
1.0,0.600863,0.225751,0.276345,0.190546,0.244675
7.0,0.273410,0.213910,0.221736,0.215688,0.220088
14.0,0.171813,0.156189,0.160739,0.150394,0.153418
30.0,0.144312,0.161042,0.122952,0.115283,0.107180


In [ ]:
print(flag)
